# Exploring City Benchmarking

This analysis looks at the City of San Diego's benchmarking program.

  1. The [city program](https://www.sandiego.gov/sustainability/energy-and-water-efficiency/benchmark) requires buildings over 50K square feet to use the EPA's Portfolio Manager to report energy use.
  2. There is an excel workbook linked off of the [who's required page](https://www.sandiego.gov/sustainability/energy-and-water-efficiency/benchmark/apply).
  3. Finally, the [published a map](https://sandiego.maps.arcgis.com/apps/webappviewer/index.html?id=ec12e750ffdf49c688852d325b299e7b) with results from the first two years (2018 and 2019).

In [1]:
import warnings
warnings.filterwarnings('ignore')
%run start.py

## Data for the Analysis

There are three different data sources. 

  1. I downloaded the csv files from the two mapped reporting years.
  2. The xlsx file is the down load from the list of buildings required to report.

In [ ]:
benchmark_2019_df = pd.read_csv('../data/2019_features.csv')
benchmark_2018_df = pd.read_csv('../data/2018_features.csv')

sd_buildings_df = pd.read_excel('../data/sdcbl_-_public_commerical_mf.xlsx',
                               sheet_name=None)

# SD Buildings Over 50K

This section builds one data set from the xlsx file.

  - I massaged a couple of the data sets to remove the first three rows (could also do this programmatically)
  - I added a square foot column if the workbook did not have one (want a consistant data frame)
  - I am using my internal version of nominatim
  - This section combines, cleans, renames, geocodes, and saves results as a shape file


## Excel Workbook Analysis

Note: There is hidden sheet.  It seems to have work in progress, so I exclude it.

Steps:

1. Read the excel with pandas
2. Result is a dictionary structure named by the worksheet with dataframe as the value
3. A couple of the sheets have a hidden column so I exclude
4. Concatenate the four separate into one
5. Rename the columns

In [ ]:
sd_buildings_df.keys()

In [ ]:
commercial_df = sd_buildings_df['Commerical']

multifamily_df = sd_buildings_df['Multifamily']

added_2020_df = sd_buildings_df['Additional Bldgs (Added 2020)']

added_2021_df = sd_buildings_df['Additional Bldgs (Added 2021)']

In [ ]:
commercial_df.columns

In [ ]:
col_names = commercial_df.columns.to_list()[:5]

In [ ]:
commercial_df = commercial_df[col_names]
multifamily_df = multifamily_df[col_names]
added_2020_df = added_2020_df[col_names]
added_2021_df = added_2021_df[col_names]

In [ ]:
commercial_df.columns

In [ ]:
column_name_map = {'Building ID #': 'BuildingID',
                  'Building Address': 'Address',
                  'City': 'City',
                  'Zip': 'Zip',
                  'Sq. ft. Building Area': 'SqFt'}

In [ ]:
commercial_df.rename(columns=column_name_map, inplace=True)
multifamily_df.rename(columns=column_name_map, inplace=True)
added_2020_df.rename(columns=column_name_map, inplace=True)
added_2021_df.rename(columns=column_name_map, inplace=True)

In [ ]:
commercial_df.columns

In [ ]:
sd_dfs = [commercial_df, multifamily_df, added_2020_df, added_2021_df]

In [ ]:
combined_df = pd.concat(sd_dfs)

In [ ]:
len(combined_df)

In [ ]:
combined_df.info()

## Geocoding

For this notebook I'm using a local version of nominatim.  I am going to comment out the specifics for a public version.

The last cell in this section writes the geocoded frame as a shape file.

In [ ]:
def clean_address(address):
    street_number, second_address = address.split('-')
    street_name = second_address.split(' ')
    first_address = " ".join([street_number,] + street_name[1:])
    return(first_address, second_address)
                             
    
def address_range(address):
    return (len (address.split('-'))) == 2

In [ ]:
# if you have an instance of nominatim running locally uncomment this to use the local
#ox.utils.config(nominatim_endpoint='http://localhost/nominatim/')

In [ ]:
import time
def geocode_address(row):
    """
    Special function applied to a zip_code transformed row.  
    
    Notes:
      1. nominatim terms of use require one query per sec so we sleep on each iteration.
      2. When we get no match, returning None so we can query for
         invalid geo's later.
    """
    time.sleep(1)
    address = row['Address']
    if address_range(address):
        first_address, second_address = clean_address(row['Address'])
        address = first_address
        
    #print(row['Building ID #'])
    geocode_query = f"{address}, {row['City']}, {str(row['Zip'])[:5]}"
    #print(geocode_query)
    try:
        #geocode_query = f"{row['Building Address']}, {row['City']}, {str(row['Zip'])[:5]}"
        lat, lon = ox.geocode(geocode_query)
        return Point(lon, lat)
    except:
        #print(row['BuildingID'])
        return None #Point(lon, lat).wkt

In [ ]:
tqdm.pandas()

In [ ]:
combined_df['geometry'] = combined_df.progress_apply(lambda r: geocode_address(r), axis=1)

In [ ]:
combined_df['geometry'].isnull().sum()

In [ ]:
_ / len(combined_df)

In [ ]:
combined_gdf = GeoDataFrame(combined_df,
                           geometry=combined_df['geometry'])

In [ ]:
combined_gdf.to_file('../data/sd_buildings.shp')

# Benchmarked Buildings

 - There are two csv files (2018 and 2019)
 - Geometry is specific to arcgis web
 - I re-projected to lat/long for map display
 - Simple example of creating a map presentation for these dataframes
 - I did not write to shape file (exercise for the reader!)

In [ ]:
benchmark_2019_df.columns

In [ ]:
benchmark_2018_df.columns

In [ ]:
info_2019 = Output(layout={'border': '1px solid black',
                            'width': '50%'})

In [ ]:
info_2018 = Output(layout={'border': '1px solid black',
                            'width': '50%'})

In [ ]:
with info_2019:
    display(HTML('<center><b>2019 Benchmark info()</b></center>'))
    display(benchmark_2019_df.info())

In [ ]:
with info_2018:
    display(HTML('<center><b>2018 Benchmark info()</b></center>'))
    display(benchmark_2018_df.info())

### df.info() side by side

In [ ]:
HBox([info_2018, info_2019])

In [ ]:
def clear_info_output():
    info_2018.clear_output()
    info_2019.clear_output()

In [ ]:
#clear_info_output()

Using receipe from https://gis.stackexchange.com/questions/276940/re-projecting-lat-and-long-in-python-geopandas-but-geometry-unchanged

In [ ]:
import pyproj
from functools import partial
from shapely.ops import transform
project = partial(
      pyproj.transform,
      pyproj.Proj('epsg:3857'), # source coordinate system
      pyproj.Proj(init='epsg:4326')) #init='epsg:4326'))

In [ ]:
benchmark_2019_gdf = GeoDataFrame(benchmark_2019_df,
                                  geometry=[transform(project, Point(xy)) for xy in zip(benchmark_2019_df.x, benchmark_2019_df.y)])

In [ ]:
benchmark_2018_gdf = GeoDataFrame(benchmark_2018_df,
                                  geometry=[transform(project, Point(xy)) for xy in zip(benchmark_2018_df.x, benchmark_2018_df.y)])

In [ ]:
benchmark_2019_gdf = benchmark_2019_gdf.rename(columns={'PropertyGFA_CalculatedBuildingsSqFt': 'AreaSqFt'})

In [ ]:
benchmark_2018_gdf['PropertyName'].fillna('Unnamed', inplace=True)
benchmark_2019_gdf['PropertyName'].fillna('Unnamed', inplace=True)

In [ ]:
import numpy as np
def msg_output(row):
    
    if np.isnan(row.BuildingID):
        bld_msg = 'UNK'
    else:
        bld_msg = int(row.BuildingID)
        
    if np.isnan(row.ENERGY_STAR_Score):
        score = 'No Score'
    else:
        score = int(row.ENERGY_STAR_Score)
        
    if np.isnan(row.AreaSqFt):
        sq_ft = "None Recorded"
    else:
        sq_ft = int(row.AreaSqFt)
        
    return HTML(f"BuildingID: {bld_msg}</br>\
                Name: {row.PropertyName}</br>\
                Address: {row.PropertyAddress}</br>\
                SqFt: {sq_ft}</br>\
                ENERGY STAR: {score}")

In [ ]:
imagery = basemap_to_tiles(basemaps.Esri.WorldImagery)
imagery.base = True
osm = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
osm.base = True


map_display = Map(center=(32.715, -117.1625), zoom=12,
                  layers=[imagery, osm],
                  layout=Layout(height="900px"),
                  scroll_wheel_zoom=True)

map_display.add_control(LayersControl())
map_display

In [ ]:
benchmark_mapped = benchmark_2019_gdf[benchmark_2019_gdf.geometry.notnull()].reset_index()

props = list()
for i, r in tqdm(benchmark_mapped.iterrows()):
    marker = CircleMarker(location=(r.geometry.y, r.geometry.x), radius=4, stroke=False, fill_color="purple", fill_opacity=1.0)#, name=r.sector)
    msg = msg_output(r) #HTML(msg_output(r))
    #msg.value = msg_output(r)
    marker.popup = msg
    props.append(marker)
    benchmark_mapped.iloc[i]['marker'] = marker
benchmark_mapped['marker'] = props

In [ ]:
property_layer = LayerGroup(name=f"2019 Benchmark ({len(props)})", layers=props)

map_display.add_layer(property_layer)#(LayerGroup(name=f"Apartments ({len(apartments)})", layers=apartments))

In [ ]:
benchmark_mapped_2018 = benchmark_2018_gdf[benchmark_2018_gdf.geometry.notnull()].reset_index()

props_2018 = list()
for i, r in tqdm(benchmark_mapped_2018.iterrows()):
    marker = CircleMarker(location=(r.geometry.y, r.geometry.x), radius=4, stroke=False, fill_color="#fa8775", fill_opacity=1.0)#, name=r.sector)
    msg = msg_output(r) #HTML(msg_output(r))
    #msg.value = msg_output(r)
    marker.popup = msg
    props_2018.append(marker)
    benchmark_mapped_2018.iloc[i]['marker'] = marker
benchmark_mapped_2018['marker'] = props_2018

In [ ]:
property_layer_2018 = LayerGroup(name=f"2018 Benchmark ({len(props)})", layers=props_2018)

map_display.add_layer(property_layer_2018)

In [ ]:
legend = LegendControl({'2019 Benchmark  ': 'purple',
                        '2018 Benchmark ': '#fa8775'}, name='Legend', position='bottomright')

map_display.add_control(legend)

### At this point, go check the map.

# Analysis Ideas for Benchmark Data Set

In [ ]:
building_id_2019 = set(benchmark_2019_df['BuildingID'])
building_id_2018 = set(benchmark_2018_df['BuildingID'])

In [ ]:
buildings_in_both = building_id_2018.intersection(building_id_2019)

In [ ]:
len(buildings_in_both)

## So, 497 buildings reported both years.

Note this was done using BuildingID and there are some missing in 2018!

## Here's a side by side looking at the CPNAME (Community Planning Group?)

In [ ]:
vc_2019 = Output(layout={'border': '1px solid black',
                            'width': '50%'})

with vc_2019:
    display(HTML('<center><b>2019 CPNAME value_counts()</b></center>'))
    display(benchmark_2019_df['CPNAME'].value_counts())
    
    
vc_2018 = Output(layout={'border': '1px solid black',
                            'width': '50%'})

with vc_2018:
    display(HTML('<center><b>2018 CPNAME value_counts()</b></center>'))
    display(benchmark_2018_df['CPNAME'].value_counts())
    
HBox([vc_2018, vc_2019])

# Finally

Here's some python code that will save the processed files as shape files.

Note there is a limit on variable names to 10 characters for the ESRI driver, so you may want to think about mapping names for the benchmarked dataframes.

```python
combined_gdf = GeoDataFrame(combined_df,
                            geometry=combined_df['geometry'])
combined_gdf.to_file('<your-file-name>.shp')

benchmark_2018_gdf.to_file('<your-file-name>.shp')
benchmark_2019_gdf.to_file('<your-file-name>.shp')                           
```